<a href="https://colab.research.google.com/github/Valgueiro/Algoritmos/blob/master/lista1/Lista_1_Modelagem_e_Monitoramento_de_sistemas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pegando os dados do github obtidos durante a fase de testes

In [6]:
import requests
import pandas as pd

url = 'https://raw.githubusercontent.com/Valgueiro/modelagem/main/lista1/log-lista1.csv?raw=true'
data = pd.read_csv(url, index_col=0, sep=';')
data.head()

,cpu_per,cpu_core,read_per_s,ram_used,ram_free,ram_buff_cache
time,,,,,,
17:28:16,5.24,2,218.55,4051,8826,2973
17:28:21,5.24,2,218.17,4062,8825,2963
17:28:26,5.23,1,217.82,4057,8821,2972
17:28:31,5.24,3,217.45,4113,8739,2998
17:28:36,5.27,1,217.05,4115,8715,3021
